# 3D Segmentation with Napari + Cellpose

This notebook shows how to:
- Load the workshop volume from `data/Lund.tif`
- Run Cellpose on the 3D stack
- Visualize the masks in Napari (3D)

## Prerequisites
- Environment managed by Pixi (see `pixi.toml`).
- Packages: `bioio`, `cellpose`, `napari`.

Install missing packages via Pixi from the repo root:

```pwsh
pixi add bioio cellpose
```

Notes:
- On macOS/Linux, `curl` is available by default; on Windows, use recent builds or PowerShell `Invoke-WebRequest` if `curl` is unavailable.
- BioIO will be used to read the 3D image.

In [1]:
from bioio import BioImage

In [2]:
image_handle = BioImage("../data/lund1051_resampled.tif")

In [3]:
import napari

# Use Dask for lazy loading of the heavy 3D image
# BioImage's .dask_data provides a Dask array without loading into memory
image_data = image_handle.dask_data.squeeze()
image_data

dask.array<getitem, shape=(213, 710, 355), dtype=float32, chunksize=(213, 710, 355), chunktype=numpy.ndarray>

In [4]:
# Create a Napari viewer and add the 3D image
viewer = napari.Viewer()
viewer.add_image(image_data, name='Lund volume')
napari.run()

In [4]:
from cellpose import io, models, core

# Initialize Cellpose model with GPU enabled
model = models.CellposeModel(gpu=True)

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

2025-12-22 16:32:53,611 [INFO] WRITING LOG OUTPUT TO C:\Users\corba\.cellpose\run.log
2025-12-22 16:32:53,612 [INFO] 
cellpose version: 	4.0.8 
platform:       	win32 
python version: 	3.11.14 
torch version:  	2.9.1
2025-12-22 16:32:53,614 [INFO] ** TORCH CUDA version installed and working. **


In [5]:
# Run Cellpose segmentation in 3D mode
masks, flows, styles = model.eval(
    image_data.compute(),
    diameter=30,  # Adjust based on typical cell size in pixels
    z_axis=0,  # Specify the z-axis for 3D data
    do_3D=True  # Enable 3D segmentation
)

2025-12-22 16:32:57,261 [INFO] running YX: 213 planes of size (710, 355)
2025-12-22 16:34:34,064 [INFO] 100%|##########| 213/213 [01:36<00:00,  2.20it/s]
2025-12-22 16:34:34,770 [INFO] running ZY: 710 planes of size (213, 355)
2025-12-22 16:36:10,583 [INFO] 100%|##########| 178/178 [01:35<00:00,  1.86it/s]
2025-12-22 16:36:11,266 [INFO] running ZX: 355 planes of size (213, 710)
2025-12-22 16:37:39,830 [INFO] 100%|##########| 178/178 [01:28<00:00,  2.01it/s]
2025-12-22 16:37:41,115 [INFO] network run in 283.85s
2025-12-22 16:37:47,122 [INFO] masks created in 4.03s


In [ ]:
viewer = napari.Viewer()
viewer.add_image(image_data, name="Lund volume", rendering="attenuated_mip")
viewer.add_labels(masks, name="Cellpose masks", opacity=0.5)
viewer.dims.ndisplay = 3
napari.run()

2025-12-22 14:09:52,224 [WARNING] DirectWrite: CreateFontFaceFromHDC() failed (Indicates an error in an input file such as a font file.) for QFontDef(Family="MS Sans Serif", pointsize=12, pixelsize=16, styleHint=5, weight=400, stretch=100, hintingPreference=0) LOGFONT("MS Sans Serif", lfWidth=0, lfHeight=-16) dpi=96


2025-12-22 14:09:56,056 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
